In [ ]:
TEST 1: NHL API - Today's Schedule

In [1]:
"""
TEST 1: NHL API - Today's Schedule
What we get: Game IDs, teams playing, start times
"""
import requests
import pandas as pd
from datetime import datetime

print("=" * 60)
print("NHL API: SCHEDULE TEST")
print("=" * 60)

# Get today's schedule
url = "https://api-web.nhle.com/v1/schedule/now"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    
    games = []
    if 'gameWeek' in data:
        for day in data['gameWeek']:
            if 'games' in day:
                for game in day['games']:
                    games.append({
                        'game_id': game.get('id'),
                        'away_team': game.get('awayTeam', {}).get('abbrev'),
                        'home_team': game.get('homeTeam', {}).get('abbrev'),
                        'start_time': game.get('startTimeUTC'),
                        'game_state': game.get('gameState')
                    })
    
    df = pd.DataFrame(games)
    print(f"\n✅ Found {len(df)} games today\n")
    print(df.to_string(index=False))
    
    # Save for later use
    df.to_csv('nhl_schedule_today.csv', index=False)
    print(f"\n💾 Saved to: nhl_schedule_today.csv")
else:
    print(f"❌ Failed: {response.status_code}")

NHL API: SCHEDULE TEST

✅ Found 42 games today

   game_id away_team home_team           start_time game_state
2025020575       PIT       TOR 2025-12-23T21:00:00Z        OFF
2025020579       DAL       DET 2025-12-23T23:30:00Z        OFF
2025020576       NYR       WSH 2025-12-24T00:00:00Z        OFF
2025020577       MTL       BOS 2025-12-24T00:00:00Z        OFF
2025020578       BUF       OTT 2025-12-24T00:00:00Z        OFF
2025020580       NJD       NYI 2025-12-24T00:00:00Z        OFF
2025020581       FLA       CAR 2025-12-24T00:00:00Z        OFF
2025020582       NSH       MIN 2025-12-24T01:00:00Z        OFF
2025020583       PHI       CHI 2025-12-24T02:00:00Z        OFF
2025020584       UTA       COL 2025-12-24T02:00:00Z        OFF
2025020585       CGY       EDM 2025-12-24T02:00:00Z        OFF
2025020586       SJS       VGK 2025-12-24T03:00:00Z        OFF
2025020587       SEA       LAK 2025-12-24T03:00:00Z        OFF
2025020588       NYR       NYI 2025-12-27T23:00:00Z        FUT
2025020

In [ ]:
TEST 2: NHL API - Player Stats

In [2]:
"""
TEST 2: NHL API - Player Stats
What we get: Current season stats for individual players
"""
import requests
import json

print("=" * 60)
print("NHL API: PLAYER STATS TEST")
print("=" * 60)

# Test with Connor McDavid (8478402)
player_id = 8478402
url = f"https://api-web.nhle.com/v1/player/{player_id}/landing"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    
    print(f"\n✅ Player: {data.get('firstName', {}).get('default')} {data.get('lastName', {}).get('default')}")
    print(f"   Team: {data.get('currentTeamAbbrev')}")
    
    if 'featuredStats' in data:
        stats = data['featuredStats']['regularSeason']['subSeason']
        
        print(f"\n📊 2025-26 Season Stats:")
        print(f"   Games: {stats.get('gamesPlayed')}")
        print(f"   Goals: {stats.get('goals')}")
        print(f"   Assists: {stats.get('assists')}")
        print(f"   Points: {stats.get('points')}")
        print(f"   PP Goals: {stats.get('powerPlayGoals')}")
        print(f"   Shots: {stats.get('shots')}")
        
        print(f"\n✅ NHL API provides: Season totals, PP stats")
        print(f"❌ NHL API does NOT provide: xG, PP1/PP2 designation, rolling averages")
else:
    print(f"❌ Failed: {response.status_code}")

NHL API: PLAYER STATS TEST

✅ Player: Connor McDavid
   Team: EDM

📊 2025-26 Season Stats:
   Games: 38
   Goals: 23
   Assists: 44
   Points: 67
   PP Goals: 7
   Shots: 128

✅ NHL API provides: Season totals, PP stats
❌ NHL API does NOT provide: xG, PP1/PP2 designation, rolling averages


In [ ]:
TEST 3: MoneyPuck CSV Analysis

In [5]:
"""
TEST 3 FIXED: MoneyPuck Analysis
Works with original MoneyPuck column names
"""
import pandas as pd

print("=" * 60)
print("MONEYPUCK: DATA ANALYSIS (FIXED)")
print("=" * 60)

# Load ORIGINAL MoneyPuck CSV (not our scraped one)
# Update this path to your actual MoneyPuck file
moneypuck_file = "data/raw/skaters.csv"  # UPDATE THIS PATH

try:
    df = pd.read_csv(moneypuck_file)
    
    print(f"\n✅ Loaded: {len(df)} rows")
    print(f"📊 Unique players: {df['name'].nunique()}")
    
    # Check season
    print(f"\n📅 Season in data: {df['season'].unique()}")
    print(f"⚠️ Is this 2025-26? Need to verify!")
    
    # Check situations
    if 'situation' in df.columns:
        print(f"\n📊 Situations available:")
        print(df['situation'].value_counts())
        
        # Filter to "all" situation for analysis
        df_all = df[df['situation'] == 'all'].copy()
        print(f"\n✅ Filtered to 'all' situation: {len(df_all)} players")
    else:
        df_all = df
    
    # MoneyPuck uses different column names
    print(f"\n📝 Sample player (top scorer in MoneyPuck data):")
    
    # MoneyPuck columns: I_F_goals, I_F_xGoals, etc.
    if 'I_F_goals' in df_all.columns:
        top_player = df_all.nlargest(1, 'I_F_goals').iloc[0]
        print(f"   Name: {top_player['name']}")
        print(f"   Team: {top_player['team']}")
        print(f"   Goals: {top_player['I_F_goals']}")
        print(f"   Games: {top_player['games_played']}")
        print(f"   xGoals: {top_player['I_F_xGoals']:.2f}")
        print(f"   Shots: {top_player['I_F_shotsOnGoal']}")
        print(f"   High Danger xG: {top_player['I_F_highDangerxGoals']:.2f}")
    
    # Check if we have PP data
    df_pp = df[df['situation'] == '5on4']
    if len(df_pp) > 0:
        print(f"\n✅ PP (5on4) data available: {len(df_pp)} players")
        print(f"   Can use this to identify PP1 players!")
        
        # Show top PP scorers
        top_pp = df_pp.nlargest(5, 'I_F_goals')[['name', 'team', 'I_F_goals', 'icetime']]
        print(f"\n🔥 Top 5 PP Goal Scorers:")
        print(top_pp.to_string(index=False))
    
    # Key question: Is this current season?
    print(f"\n" + "=" * 60)
    print(f"🔍 DATA FRESHNESS CHECK:")
    print(f"=" * 60)
    
    # Check Connor McDavid's stats
    mcdavid = df_all[df_all['name'].str.contains('McDavid', case=False, na=False)]
    if len(mcdavid) > 0:
        mcd = mcdavid.iloc[0]
        print(f"\nConnor McDavid in MoneyPuck:")
        print(f"  Season: {mcd['season']}")
        print(f"  Games: {mcd['games_played']}")
        print(f"  Goals: {mcd['I_F_goals']}")
        
        print(f"\nConnor McDavid in NHL API (from Test 2):")
        print(f"  Season: 2025-26")
        print(f"  Games: 38")
        print(f"  Goals: 23")
        
        if mcd['games_played'] == 38 and mcd['I_F_goals'] == 23:
            print(f"\n✅ MATCH! MoneyPuck data IS current (2025-26)")
        else:
            print(f"\n❌ MISMATCH! MoneyPuck data is OLD")
            print(f"   MoneyPuck shows: {mcd['games_played']} GP, {mcd['I_F_goals']} G")
            print(f"   Current season: 38 GP, 23 G")
    
    print(f"\n✅ MoneyPuck provides:")
    print(f"   - Advanced xG metrics (I_F_xGoals, high/med/low danger)")
    print(f"   - Situation splits (5on5, PP, PK)")
    print(f"   - Historical depth")
    
    print(f"\n⚠️ MoneyPuck limitations:")
    print(f"   - May lag 1-2 days")
    print(f"   - Need to verify season/freshness")
    print(f"   - No clean PP1/PP2 labels (must infer from icetime)")

except FileNotFoundError:
    print(f"❌ File not found: {moneypuck_file}")
    print(f"💡 Update the file path to your actual MoneyPuck CSV")
except Exception as e:
    print(f"❌ Error: {e}")

MONEYPUCK: DATA ANALYSIS (FIXED)

✅ Loaded: 4135 rows
📊 Unique players: 826

📅 Season in data: [2025]
⚠️ Is this 2025-26? Need to verify!

📊 Situations available:
situation
other    827
all      827
5on5     827
4on5     827
5on4     827
Name: count, dtype: int64

✅ Filtered to 'all' situation: 827 players

📝 Sample player (top scorer in MoneyPuck data):
   Name: Nathan MacKinnon
   Team: COL
   Goals: 30.0
   Games: 35
   xGoals: 19.04
   Shots: 161.0
   High Danger xG: 8.17

✅ PP (5on4) data available: 827 players
   Can use this to identify PP1 players!

🔥 Top 5 PP Goal Scorers:
           name team  I_F_goals  icetime
 Wyatt Johnston  DAL       11.0   6838.0
 Alex DeBrincat  DET        9.0   7513.0
Pavel Dorofeyev  VGK        9.0   7140.0
 Leon Draisaitl  EDM        8.0   7306.0
   Jake DeBrusk  VAN        8.0   7357.0

🔍 DATA FRESHNESS CHECK:

Connor McDavid in MoneyPuck:
  Season: 2025
  Games: 37
  Goals: 23.0

Connor McDavid in NHL API (from Test 2):
  Season: 2025-26
  Games: 

In [6]:
"""
QUICK VERIFICATION: Compare MoneyPuck vs NHL API
"""
import pandas as pd

print("=" * 60)
print("MONEYPUCK vs NHL API - FRESHNESS CHECK")
print("=" * 60)

# Load both datasets
moneypuck = pd.read_csv("data/raw/skaters.csv")  # UPDATE PATH
nhl_scraped = pd.read_csv("data/raw/nhl_2025_26_current_season.csv")

# Filter MoneyPuck to 'all' situation
mp_all = moneypuck[moneypuck['situation'] == 'all'].copy()

# Test players
test_players = ['Connor McDavid', 'Nathan MacKinnon', 'Auston Matthews']

print("\nComparison for key players:\n")

for player_name in test_players:
    # MoneyPuck data
    mp_player = mp_all[mp_all['name'].str.contains(player_name, case=False, na=False)]
    
    # NHL API data
    nhl_player = nhl_scraped[nhl_scraped['name'].str.contains(player_name, case=False, na=False)]
    
    if len(mp_player) > 0 and len(nhl_player) > 0:
        mp = mp_player.iloc[0]
        nhl = nhl_player.iloc[0]
        
        print(f"{player_name}:")
        print(f"  MoneyPuck: {mp['games_played']} GP, {mp['I_F_goals']} G")
        print(f"  NHL API:   {nhl['games_played']} GP, {nhl['goals']} G")
        
        if mp['games_played'] == nhl['games_played']:
            print(f"  ✅ SAME - MoneyPuck is current!")
        else:
            print(f"  ❌ DIFFERENT - MoneyPuck is outdated!")
        print()

print("\n" + "=" * 60)
print("CONCLUSION:")
print("=" * 60)
print("\nIf games_played match → Use MoneyPuck for advanced xG")
print("If games_played differ → Stick with NHL API scraped data")

MONEYPUCK vs NHL API - FRESHNESS CHECK

Comparison for key players:

Connor McDavid:
  MoneyPuck: 37 GP, 23.0 G
  NHL API:   37 GP, 23 G
  ✅ SAME - MoneyPuck is current!

Nathan MacKinnon:
  MoneyPuck: 35 GP, 30.0 G
  NHL API:   35 GP, 30 G
  ✅ SAME - MoneyPuck is current!

Auston Matthews:
  MoneyPuck: 30 GP, 14.0 G
  NHL API:   30 GP, 14 G
  ✅ SAME - MoneyPuck is current!


CONCLUSION:

If games_played match → Use MoneyPuck for advanced xG
If games_played differ → Stick with NHL API scraped data


In [ ]:
TEST 4: Natural Stat Trick - Individual Stats

In [7]:
"""
TEST 4: Natural Stat Trick - Individual Stats
What we can get: ixG, high-danger chances, situation splits
"""
import requests
from bs4 import BeautifulSoup
import pandas as pd

print("=" * 60)
print("NATURAL STAT TRICK: SCRAPING TEST")
print("=" * 60)

# NST individual stats page
url = "https://www.naturalstattrick.com/playerteams.php?fromseason=20252026&thruseason=20252026&stype=2&sit=all&score=all&stdoi=std&rate=n&team=ALL&pos=S&loc=B&toi=0&gpfilt=none&fd=&td=&tgp=410&lines=single"

try:
    response = requests.get(url, timeout=10)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the stats table
        table = soup.find('table', {'id': 'teams'})
        
        if table:
            print("✅ NST page accessible")
            print("✅ Table found")
            
            # Get headers
            headers = [th.text.strip() for th in table.find_all('th')]
            print(f"\n📊 Available columns ({len(headers)}):")
            print(headers[:20])  # Show first 20
            
            print(f"\n✅ NST provides: ixG, xGF, CF%, shots, high-danger chances")
            print(f"⚠️ NST requires: Web scraping or manual CSV export")
            print(f"💡 Alternative: Export CSV manually from NST website")
        else:
            print("❌ Could not find table - page structure may have changed")
    else:
        print(f"❌ Failed: {response.status_code}")
        
except Exception as e:
    print(f"❌ Error: {e}")
    print("\n💡 NST might require manual CSV export instead of scraping")

NATURAL STAT TRICK: SCRAPING TEST
❌ Could not find table - page structure may have changed


In [ ]:
TEST 5: DailyFaceoff - PP Lines

In [8]:
"""
TEST 5: DailyFaceoff - PP Lines
What we can get: PP1/PP2 units, starting goalies
"""
import requests
from bs4 import BeautifulSoup

print("=" * 60)
print("DAILYFACEOFF: PP UNITS TEST")
print("=" * 60)

# Test with one team (Toronto)
team = "toronto-maple-leafs"
url = f"https://www.dailyfaceoff.com/teams/{team}/line-combinations/"

try:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    response = requests.get(url, headers=headers, timeout=10)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        print(f"✅ DailyFaceoff accessible for {team}")
        
        # Look for PP section (structure varies)
        pp_section = soup.find('div', class_=lambda x: x and 'powerplay' in x.lower() if x else False)
        
        if pp_section:
            print("✅ PP section found")
            print("\n📝 Sample HTML structure:")
            print(str(pp_section)[:500])
        else:
            # Try alternate structure
            print("\n🔍 Searching for player names...")
            players = soup.find_all('a', class_=lambda x: x and 'player' in x.lower() if x else False)
            if players:
                print(f"✅ Found {len(players)} player links")
                print("Sample players:", [p.text.strip() for p in players[:5]])
        
        print(f"\n✅ DailyFaceoff provides: PP1/PP2, line combos, goalies")
        print(f"⚠️ DailyFaceoff requires: Web scraping (HTML structure)")
        print(f"💡 Alternative: Manual entry for 32 teams (5 players per PP1)")
        
    else:
        print(f"❌ Failed: {response.status_code}")
        
except Exception as e:
    print(f"❌ Error: {e}")

DAILYFACEOFF: PP UNITS TEST
✅ DailyFaceoff accessible for toronto-maple-leafs

🔍 Searching for player names...

✅ DailyFaceoff provides: PP1/PP2, line combos, goalies
⚠️ DailyFaceoff requires: Web scraping (HTML structure)
💡 Alternative: Manual entry for 32 teams (5 players per PP1)


In [ ]:
TEST 6: OddsAPI - Player Props Test

In [9]:
"""
TEST 6: OddsAPI - Player Props Test
What we need: Anytime goal scorer odds
"""
import requests
from datetime import datetime

print("=" * 60)
print("ODDSAPI: PLAYER PROPS TEST")
print("=" * 60)

API_KEY = "b81efa2d2e27c141b788a7fb02065fe0"  # UPDATE THIS

# Current time check
now = datetime.now()
print(f"\n⏰ Current time: {now.strftime('%Y-%m-%d %H:%M:%S')}")

# Get first game start time from schedule
schedule_url = "https://api-web.nhle.com/v1/schedule/now"
schedule_data = requests.get(schedule_url).json()

first_game_time = None
if 'gameWeek' in schedule_data:
    for day in schedule_data['gameWeek']:
        if 'games' in day and day['games']:
            first_game_time = day['games'][0].get('startTimeUTC')
            break

if first_game_time:
    print(f"🏒 First game starts: {first_game_time}")
    print(f"💡 Player props typically appear 1-4 hours before game time")

# Test OddsAPI
url = "https://api.the-odds-api.com/v4/sports/icehockey_nhl/odds"

markets_to_test = [
    'player_goal_scorer',
    'player_anytime_goalscorer',
    'player_anytime_goal_scorer'
]

print(f"\n🧪 Testing markets...\n")

for market in markets_to_test:
    params = {
        'apiKey': API_KEY,
        'regions': 'us',
        'markets': market,
        'oddsFormat': 'decimal'
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        
        # Check if we have player outcomes
        has_props = False
        if data and 'bookmakers' in data[0]:
            for bookmaker in data[0].get('bookmakers', []):
                if 'markets' in bookmaker and bookmaker['markets']:
                    for market_data in bookmaker['markets']:
                        if 'outcomes' in market_data and market_data['outcomes']:
                            has_props = True
                            print(f"✅ {market}: FOUND!")
                            print(f"   Events: {len(data)}")
                            print(f"   Sample player: {market_data['outcomes'][0].get('description')}")
                            print(f"   Sample odds: {market_data['outcomes'][0].get('price')}")
                            break
        
        if not has_props:
            print(f"❌ {market}: No player props available")
    else:
        print(f"❌ {market}: Failed ({response.status_code})")

print(f"\n📊 Requests remaining: {response.headers.get('x-requests-remaining', 'Unknown')}")

print(f"\n💡 RECOMMENDATIONS:")
print(f"   - If no props: Try again 2-3 hours before game time")
print(f"   - If still fails: Use PrizePicks or manual entry")
print(f"   - OddsAPI tier: Check if player props included in your plan")

ODDSAPI: PLAYER PROPS TEST

⏰ Current time: 2025-12-24 14:29:25
🏒 First game starts: 2025-12-23T21:00:00Z
💡 Player props typically appear 1-4 hours before game time

🧪 Testing markets...

❌ player_goal_scorer: Failed (422)
❌ player_anytime_goalscorer: Failed (422)
❌ player_anytime_goal_scorer: Failed (422)

📊 Requests remaining: 408

💡 RECOMMENDATIONS:
   - If no props: Try again 2-3 hours before game time
   - If still fails: Use PrizePicks or manual entry
   - OddsAPI tier: Check if player props included in your plan


In [ ]:
TEST 7: Data Source Summary

In [10]:
"""
TEST 7: Data Source Summary
Compare what each source provides
"""
import pandas as pd

print("=" * 80)
print("DATA SOURCE COMPARISON MATRIX")
print("=" * 80)

comparison = pd.DataFrame({
    'Source': ['NHL API', 'MoneyPuck CSV', 'Natural Stat Trick', 'DailyFaceoff', 'OddsAPI'],
    'Current Season Stats': ['✅ Yes', '✅ Yes', '✅ Yes (scrape)', '❌ No', '❌ No'],
    'xG / ixG': ['❌ No', '✅ Yes (historical)', '✅ Yes (current)', '❌ No', '❌ No'],
    'PP1/PP2 Labels': ['❌ No', '⚠️ Inferred', '❌ No', '✅ Yes', '❌ No'],
    'Today\'s Lineups': ['✅ Yes (rosters)', '❌ No', '❌ No', '✅ Yes', '❌ No'],
    'Betting Odds': ['❌ No', '❌ No', '❌ No', '❌ No', '✅ Maybe'],
    'Update Frequency': ['Real-time', 'Daily lag', 'Daily', 'Daily', 'Live'],
    'Access': ['Free API', 'Free CSV', 'Free (scrape)', 'Free (scrape)', 'Paid API'],
    'Best Used For': ['Lineup gate', 'xG priors', 'Rolling stats', 'PP units', 'Market odds']
})

print("\n")
print(comparison.to_string(index=False))

print("\n" + "=" * 80)
print("RECOMMENDED DATA PIPELINE")
print("=" * 80)
print("""
1. NHL API → Confirm today's games & rosters (GATE)
2. Your scraped CSV → Player stats & xG estimates (PRIORS)  
3. DailyFaceoff → PP1 units (manual/scrape) (ROLE BOOST)
4. OddsAPI → Player props if available (MARKET)
   └─ Fallback: Screenshot + Claude OCR

MISSING & NEED TO ADD:
- PP1/PP2 labels (DailyFaceoff or manual)
- Live odds (OddsAPI retry or alternative)
""")

DATA SOURCE COMPARISON MATRIX


            Source Current Season Stats           xG / ixG PP1/PP2 Labels Today's Lineups Betting Odds Update Frequency        Access Best Used For
           NHL API                ✅ Yes               ❌ No           ❌ No ✅ Yes (rosters)         ❌ No        Real-time      Free API   Lineup gate
     MoneyPuck CSV                ✅ Yes ✅ Yes (historical)    ⚠️ Inferred            ❌ No         ❌ No        Daily lag      Free CSV     xG priors
Natural Stat Trick       ✅ Yes (scrape)    ✅ Yes (current)           ❌ No            ❌ No         ❌ No            Daily Free (scrape) Rolling stats
      DailyFaceoff                 ❌ No               ❌ No          ✅ Yes           ✅ Yes         ❌ No            Daily Free (scrape)      PP units
           OddsAPI                 ❌ No               ❌ No           ❌ No            ❌ No      ✅ Maybe             Live      Paid API   Market odds

RECOMMENDED DATA PIPELINE

1. NHL API → Confirm today's games & rosters (GATE)


In [11]:
"""
ODDSAPI - CORRECT MARKET NAME
Test with: player_goal_scorer_anytime
"""

import requests
import pandas as pd
from datetime import datetime

print("=" * 60)
print("ODDSAPI: ANYTIME GOAL SCORER (CORRECT MARKET)")
print("=" * 60)

API_KEY = "b81efa2d2e27c141b788a7fb02065fe0"  # UPDATE THIS

url = "https://api.the-odds-api.com/v4/sports/icehockey_nhl/odds"

params = {
    'apiKey': API_KEY,
    'regions': 'us',
    'markets': 'player_goal_scorer_anytime',  # ← THE CORRECT NAME!
    'oddsFormat': 'decimal'
}

print(f"\n⏰ Testing at: {datetime.now().strftime('%H:%M:%S')}")
print(f"🧪 Market: player_goal_scorer_anytime\n")

response = requests.get(url, params=params)

print(f"Status: {response.status_code}")
print(f"Requests remaining: {response.headers.get('x-requests-remaining')}")

if response.status_code == 200:
    data = response.json()
    
    print(f"\n✅ SUCCESS!")
    print(f"📊 Events returned: {len(data)}")
    
    if data:
        # Extract all player props
        all_props = []
        
        for event in data:
            away = event.get('away_team')
            home = event.get('home_team')
            
            if 'bookmakers' not in event:
                continue
            
            for bookmaker in event['bookmakers']:
                bookie = bookmaker.get('key')
                
                if 'markets' not in bookmaker:
                    continue
                
                for market in bookmaker['markets']:
                    if market.get('key') == 'player_goal_scorer_anytime':
                        
                        for outcome in market.get('outcomes', []):
                            all_props.append({
                                'player': outcome.get('description'),
                                'bookmaker': bookie,
                                'odds': outcome.get('price'),
                                'home_team': home,
                                'away_team': away
                            })
        
        if all_props:
            df = pd.DataFrame(all_props)
            
            print(f"\n🎯 TOTAL PLAYER PROPS: {len(df)}")
            print(f"📚 Bookmakers: {df['bookmaker'].nunique()}")
            print(f"\n📝 Sample (first 20):\n")
            print(df.head(20).to_string(index=False))
            
            # Save
            df.to_csv('oddsapi_goal_scorer_props.csv', index=False)
            print(f"\n💾 Saved to: oddsapi_goal_scorer_props.csv")
            
            # Quick stats
            print(f"\n📊 ODDS DISTRIBUTION:")
            print(df['odds'].describe())
            
            print(f"\n✅ ODDSAPI WORKS! You have {len(df)} player props!")
            
        else:
            print(f"\n⚠️ No player props found in markets")
    else:
        print(f"\n⚠️ No events returned - games may not have props yet")
        
elif response.status_code == 422:
    print(f"\n❌ 422 Error - Market name still wrong or not available")
    print(f"Response: {response.text}")
else:
    print(f"\n❌ Failed: {response.status_code}")
    print(f"Response: {response.text}")

ODDSAPI: ANYTIME GOAL SCORER (CORRECT MARKET)

⏰ Testing at: 14:39:31
🧪 Market: player_goal_scorer_anytime

Status: 422
Requests remaining: 408

❌ 422 Error - Market name still wrong or not available
Response: {"message":"Markets not supported by this endpoint: player_goal_scorer_anytime","error_code":"INVALID_MARKET","details_url":"https://the-odds-api.com/liveapi/guides/v4/api-error-codes.html#invalid-market"}



In [12]:
"""
ODDSAPI - PLAYER PROPS (CORRECT ENDPOINT)
Player props require /events/{event_id}/odds endpoint
"""

import requests
import pandas as pd

print("=" * 60)
print("ODDSAPI: ANYTIME GOAL SCORER (EVENT ENDPOINT)")
print("=" * 60)

API_KEY = "b81efa2d2e27c141b788a7fb02065fe0"  # UPDATE THIS

# Step 1: Get today's NHL events
print("\n1️⃣ Getting NHL events...\n")

events_url = "https://api.the-odds-api.com/v4/sports/icehockey_nhl/events"
params = {'apiKey': API_KEY}

events_response = requests.get(events_url, params=params)

if events_response.status_code != 200:
    print(f"❌ Failed to get events: {events_response.status_code}")
    exit()

events = events_response.json()
print(f"✅ Found {len(events)} events today\n")

# Step 2: Get player props for each event
print("2️⃣ Fetching player props for each event...\n")

all_props = []

for i, event in enumerate(events[:3], 1):  # Test with first 3 events
    event_id = event.get('id')
    away = event.get('away_team')
    home = event.get('home_team')
    
    print(f"[{i}/{min(3, len(events))}] {away} @ {home}...", end=" ")
    
    # Event-specific odds endpoint
    event_url = f"https://api.the-odds-api.com/v4/sports/icehockey_nhl/events/{event_id}/odds"
    
    params = {
        'apiKey': API_KEY,
        'regions': 'us',
        'markets': 'player_goal_scorer_anytime',
        'oddsFormat': 'decimal'
    }
    
    response = requests.get(event_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        
        if 'bookmakers' in data:
            props_found = 0
            
            for bookmaker in data['bookmakers']:
                if 'markets' not in bookmaker:
                    continue
                
                for market in bookmaker['markets']:
                    if market.get('key') == 'player_goal_scorer_anytime':
                        
                        for outcome in market.get('outcomes', []):
                            all_props.append({
                                'player': outcome.get('description'),
                                'bookmaker': bookmaker.get('key'),
                                'odds': outcome.get('price'),
                                'home_team': home,
                                'away_team': away
                            })
                            props_found += 1
            
            print(f"✅ {props_found} props")
        else:
            print("❌ No bookmakers")
    else:
        print(f"❌ {response.status_code}")

print(f"\n📊 Requests remaining: {response.headers.get('x-requests-remaining')}")

if all_props:
    df = pd.DataFrame(all_props)
    
    print(f"\n" + "=" * 60)
    print(f"✅ SUCCESS! FOUND {len(df)} PLAYER PROPS")
    print("=" * 60)
    
    print(f"\n📝 Sample (first 20):\n")
    print(df.head(20).to_string(index=False))
    
    df.to_csv('oddsapi_props_SUCCESS.csv', index=False)
    print(f"\n💾 Saved to: oddsapi_props_SUCCESS.csv")
    
    print(f"\n🎉 ODDSAPI WORKS! Next: Run full pipeline")
    
else:
    print(f"\n❌ No props found")
    print(f"\n💡 POSSIBLE REASONS:")
    print(f"   1. Props not yet available (try closer to game time)")
    print(f"   2. Free tier doesn't include player props")
    print(f"   3. NHL not supported for player props on OddsAPI")
    
    print(f"\n🔄 FALLBACK OPTIONS:")
    print(f"   A) Wait 2 hours and retry")
    print(f"   B) Screenshot + Claude OCR (2 min)")
    print(f"   C) Manual CSV entry (5 min)")
    print(f"   D) PrizePicks scraper (15 min)")

ODDSAPI: ANYTIME GOAL SCORER (EVENT ENDPOINT)

1️⃣ Getting NHL events...

✅ Found 13 events today

2️⃣ Fetching player props for each event...

[1/3] New York Rangers @ New York Islanders... ✅ 0 props
[2/3] Boston Bruins @ Buffalo Sabres... ✅ 0 props
[3/3] Detroit Red Wings @ Carolina Hurricanes... ✅ 0 props

📊 Requests remaining: 408

❌ No props found

💡 POSSIBLE REASONS:
   1. Props not yet available (try closer to game time)
   2. Free tier doesn't include player props
   3. NHL not supported for player props on OddsAPI

🔄 FALLBACK OPTIONS:
   A) Wait 2 hours and retry
   B) Screenshot + Claude OCR (2 min)
   C) Manual CSV entry (5 min)
   D) PrizePicks scraper (15 min)


In [13]:
"""
Check which games haven't started yet
"""
import requests
from datetime import datetime, timezone

url = "https://api-web.nhle.com/v1/schedule/now"
data = requests.get(url).json()

print("=" * 60)
print("NHL GAMES STATUS")
print("=" * 60)

now = datetime.now(timezone.utc)

upcoming = []
live = []
finished = []

if 'gameWeek' in data:
    for day in data['gameWeek']:
        if 'games' in day:
            for game in day['games']:
                start_time = datetime.fromisoformat(game['startTimeUTC'].replace('Z', '+00:00'))
                away = game['awayTeam']['abbrev']
                home = game['homeTeam']['abbrev']
                state = game['gameState']
                
                game_info = f"{away} @ {home} - {start_time.strftime('%H:%M UTC')}"
                
                if state == 'FUT':  # Future
                    upcoming.append(game_info)
                elif state == 'LIVE':
                    live.append(game_info)
                else:
                    finished.append(game_info)

print(f"\n⏰ Current time: {now.strftime('%Y-%m-%d %H:%M UTC')}\n")

print(f"🔜 UPCOMING GAMES ({len(upcoming)}):")
for g in upcoming:
    print(f"   {g}")

print(f"\n🔴 LIVE GAMES ({len(live)}):")
for g in live:
    print(f"   {g}")

print(f"\n✅ FINISHED GAMES ({len(finished)}):")
for g in finished[:5]:  # Just show first 5
    print(f"   {g}")

if upcoming:
    print(f"\n✅ {len(upcoming)} games still upcoming - odds should be available!")
elif live:
    print(f"\n⚠️ Only live games - odds might be limited")
else:
    print(f"\n❌ All games finished - wait for tomorrow's games")

NHL GAMES STATUS

⏰ Current time: 2025-12-24 13:57 UTC

🔜 UPCOMING GAMES (29):
   NYR @ NYI - 23:00 UTC
   BOS @ BUF - 00:00 UTC
   OTT @ TOR - 00:00 UTC
   TBL @ FLA - 00:00 UTC
   WSH @ NJD - 00:00 UTC
   DET @ CAR - 00:00 UTC
   MIN @ WPG - 00:00 UTC
   NSH @ STL - 01:00 UTC
   CHI @ DAL - 01:00 UTC
   ANA @ LAK - 02:00 UTC
   EDM @ CGY - 03:00 UTC
   SJS @ VAN - 03:00 UTC
   COL @ VGK - 03:00 UTC
   MTL @ TBL - 22:00 UTC
   NYI @ CBJ - 22:00 UTC
   TOR @ DET - 00:00 UTC
   PIT @ CHI - 00:00 UTC
   PHI @ SEA - 01:00 UTC
   CBJ @ OTT - 00:00 UTC
   WSH @ FLA - 00:00 UTC
   NYR @ CAR - 00:00 UTC
   EDM @ WPG - 00:30 UTC
   BUF @ STL - 01:00 UTC
   LAK @ COL - 02:00 UTC
   NSH @ UTA - 02:00 UTC
   BOS @ CGY - 02:00 UTC
   MIN @ VGK - 03:00 UTC
   SJS @ ANA - 03:00 UTC
   VAN @ SEA - 03:00 UTC

🔴 LIVE GAMES (0):

✅ FINISHED GAMES (13):
   PIT @ TOR - 21:00 UTC
   DAL @ DET - 23:30 UTC
   NYR @ WSH - 00:00 UTC
   MTL @ BOS - 00:00 UTC
   BUF @ OTT - 00:00 UTC

✅ 29 games still upcoming -